In [13]:
import requests
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta # Para adicionar meses facilmente

def baixar_dados_bcb_mes_a_mes(
    ano_inicio=2010,
    mes_inicio=1,
    tipo_dado="BANCOS",
    diretorio_destino="dados_bcb_cosif"
):
    """
    Baixa dados do Banco Central mês a mês, usando um padrão de URL.

    Args:
        ano_inicio (int): Ano de início para o download.
        mes_inicio (int): Mês de início para o download.
        tipo_dado (str): Sufixo no nome do arquivo (ex: "BANCOS").
        diretorio_destino (str): O diretório onde os arquivos serão salvos.
    """
    base_url = "https://www.bcb.gov.br/content/estabilidadefinanceira/cosif/{}/"

    # Cria o diretório de destino se ele não existir
    if not os.path.exists(diretorio_destino):
        os.makedirs(diretorio_destino)
        print(f"Diretório '{diretorio_destino}' criado.")
    else:
        print(f"Diretório '{diretorio_destino}' já existe.")

    data_atual = date.today()
    data_iteracao = date(ano_inicio, mes_inicio, 1)

    print(f"Iniciando download dos dados de {mes_inicio:02d}/{ano_inicio} até {data_atual.month:02d}/{data_atual.year}...")

    # Loop pelos meses
    while data_iteracao <= data_atual:
        ano = data_iteracao.year
        mes = data_iteracao.month

        # Formata o mês e ano para o URL
        mes_str = f"{mes:02d}" # Garante dois dígitos (ex: 01, 02)
        ano_mes_str = f"{ano}{mes_str}"

        # Monta a URL completa do arquivo
        # Ex: https://www.bcb.gov.br/content/estabilidadefinanceira/cosif/Bancos/202503BANCOS.zip
        download_url = base_url.format(tipo_dado) + f"{ano_mes_str}{tipo_dado}.zip"
        
        # Define o nome do arquivo local
        nome_arquivo = f"{ano_mes_str}{tipo_dado}.csv.zip"
        caminho_completo_arquivo = os.path.join(diretorio_destino, nome_arquivo)

        print(f"Tentando baixar: {nome_arquivo}")

        if os.path.exists(caminho_completo_arquivo):
            print(f"  Arquivo '{nome_arquivo}' já existe. Pulando.")
        else:
            try:
                response = requests.get(download_url, stream=True)
                response.raise_for_status() # Levanta um erro para códigos de status HTTP ruins (4xx ou 5xx)

                with open(caminho_completo_arquivo, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"  Download concluído: {nome_arquivo}")
            except requests.exceptions.RequestException as e:
                print(f"  Erro ao baixar {nome_arquivo}: {e}")
                print(f"  URL tentada: {download_url}")
                # Se um arquivo não existir, não é necessariamente um erro fatal, pode ser que o BCB
                # não o tenha disponibilizado para aquele mês/ano.

        # Avança para o próximo mês
        data_iteracao += relativedelta(months=1)

# --- Configurações de Download ---
ANO_INICIO_DOWNLOAD = 2010
MES_INICIO_DOWNLOAD = 1 # Janeiro

# O sufixo que identifica o tipo de dado na URL e no nome do arquivo.
# Pelo seu exemplo, "Bancos" é para Balancetes de Bancos.
# Se houver outros tipos de dados com padrões semelhantes,
# você pode chamar a função novamente com um tipo_dado diferente.
TIPO_DADO_PARA_BAIXAR = "Bancos"

# Chama a função para iniciar o download
baixar_dados_bcb_mes_a_mes(
    ano_inicio=ANO_INICIO_DOWNLOAD,
    mes_inicio=MES_INICIO_DOWNLOAD,
    tipo_dado=TIPO_DADO_PARA_BAIXAR
)

print("\nProcesso de download concluído.")

Diretório 'dados_bcb_cosif' já existe.
Iniciando download dos dados de 01/2010 até 07/2025...
Tentando baixar: 201001Bancos.csv.zip
  Download concluído: 201001Bancos.csv.zip
Tentando baixar: 201002Bancos.csv.zip
  Download concluído: 201002Bancos.csv.zip
Tentando baixar: 201003Bancos.csv.zip
  Download concluído: 201003Bancos.csv.zip
Tentando baixar: 201004Bancos.csv.zip
  Download concluído: 201004Bancos.csv.zip
Tentando baixar: 201005Bancos.csv.zip
  Download concluído: 201005Bancos.csv.zip
Tentando baixar: 201006Bancos.csv.zip
  Download concluído: 201006Bancos.csv.zip
Tentando baixar: 201007Bancos.csv.zip
  Download concluído: 201007Bancos.csv.zip
Tentando baixar: 201008Bancos.csv.zip
  Download concluído: 201008Bancos.csv.zip
Tentando baixar: 201009Bancos.csv.zip
  Download concluído: 201009Bancos.csv.zip
Tentando baixar: 201010Bancos.csv.zip
  Download concluído: 201010Bancos.csv.zip
Tentando baixar: 201011Bancos.csv.zip
  Download concluído: 201011Bancos.csv.zip
Tentando baixar

In [7]:
import zipfile
import os

diretorio_zips="dados_bcb_cosif"
diretorio_destino_extraidos="dados_bancos"

arquivos_no_diretorio = os.listdir("dados_bcb_cosif")
zips_encontrados = [f for f in arquivos_no_diretorio if f.endswith('.zip')]

for i, nome_zip in enumerate(zips_encontrados):
    caminho_completo_zip = os.path.join("dados_bcb_cosif", nome_zip)
    
    try:
        with zipfile.ZipFile(caminho_completo_zip, 'r') as zip_ref:
            # Extrai todo o conteúdo do zip para o diretório de destino
            zip_ref.extractall(diretorio_destino_extraidos)
            print(f"  Extraído '{nome_zip}' ({i+1}/{len(zips_encontrados)}) para '{diretorio_destino_extraidos}'.")
    except zipfile.BadZipFile:
        print(f"  Erro: '{nome_zip}' não é um arquivo zip válido ou está corrompido. Pulando.")
    except Exception as e:
        print(f"  Erro inesperado ao extrair '{nome_zip}': {e}. Pulando.")

  Extraído '202411Bancos.csv.zip' (1/181) para 'dados_bancos'.
  Extraído '202002Bancos.csv.zip' (2/181) para 'dados_bancos'.
  Extraído '201601Bancos.csv.zip' (3/181) para 'dados_bancos'.
  Extraído '201001Bancos.csv.zip' (4/181) para 'dados_bancos'.
  Extraído '202109Bancos.csv.zip' (5/181) para 'dados_bancos'.
  Extraído '202201Bancos.csv.zip' (6/181) para 'dados_bancos'.
  Extraído '202503Bancos.csv.zip' (7/181) para 'dados_bancos'.
  Extraído '201908Bancos.csv.zip' (8/181) para 'dados_bancos'.
  Extraído '201010Bancos.csv.zip' (9/181) para 'dados_bancos'.
  Extraído '201611Bancos.csv.zip' (10/181) para 'dados_bancos'.
  Extraído '201705Bancos.csv.zip' (11/181) para 'dados_bancos'.
  Extraído '201701Bancos.csv.zip' (12/181) para 'dados_bancos'.
  Extraído '202207Bancos.csv.zip' (13/181) para 'dados_bancos'.
  Extraído '202312Bancos.csv.zip' (14/181) para 'dados_bancos'.
  Extraído '201612Bancos.csv.zip' (15/181) para 'dados_bancos'.
  Extraído '201012Bancos.csv.zip' (16/181) para '

In [ ]:
def baixar_dados_bcb_mes_a_mes(
    ano_inicio=2010,
    mes_inicio=1,
    tipo_dado="Sociedades",
    diretorio_destino="dados_bcb_cosif_sociedades"
):
    """
    Baixa dados do Banco Central mês a mês, usando um padrão de URL.

    Args:
        ano_inicio (int): Ano de início para o download.
        mes_inicio (int): Mês de início para o download.
        tipo_dado (str): Sufixo no nome do arquivo (ex: "BANCOS").
        diretorio_destino (str): O diretório onde os arquivos serão salvos.
    """
    base_url = "https://www.bcb.gov.br/content/estabilidadefinanceira/cosif/{}/"

    # Cria o diretório de destino se ele não existir
    if not os.path.exists(diretorio_destino):
        os.makedirs(diretorio_destino)
        print(f"Diretório '{diretorio_destino}' criado.")
    else:
        print(f"Diretório '{diretorio_destino}' já existe.")

    data_atual = date.today()
    data_iteracao = date(ano_inicio, mes_inicio, 1)

    print(f"Iniciando download dos dados de {mes_inicio:02d}/{ano_inicio} até {data_atual.month:02d}/{data_atual.year}...")

    # Loop pelos meses
    while data_iteracao <= data_atual:
        ano = data_iteracao.year
        mes = data_iteracao.month

        # Formata o mês e ano para o URL
        mes_str = f"{mes:02d}" # Garante dois dígitos (ex: 01, 02)
        ano_mes_str = f"{ano}{mes_str}"

        # Monta a URL completa do arquivo
        # Ex: https://www.bcb.gov.br/content/estabilidadefinanceira/cosif/Bancos/202503BANCOS.zip
        download_url = base_url.format(tipo_dado) + f"{ano_mes_str}{tipo_dado}.csv.zip"
        
        # Define o nome do arquivo local
        nome_arquivo = f"{ano_mes_str}{tipo_dado}.zip"
        caminho_completo_arquivo = os.path.join(diretorio_destino, nome_arquivo)

        print(f"Tentando baixar: {nome_arquivo}")

        if os.path.exists(caminho_completo_arquivo):
            print(f"  Arquivo '{nome_arquivo}' já existe. Pulando.")
        else:
            try:
                response = requests.get(download_url, stream=True)
                response.raise_for_status() # Levanta um erro para códigos de status HTTP ruins (4xx ou 5xx)

                with open(caminho_completo_arquivo, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"  Download concluído: {nome_arquivo}")
            except requests.exceptions.RequestException as e:
                print(f"  Erro ao baixar {nome_arquivo}: {e}")
                print(f"  URL tentada: {download_url}")
                # Se um arquivo não existir, não é necessariamente um erro fatal, pode ser que o BCB
                # não o tenha disponibilizado para aquele mês/ano.

        # Avança para o próximo mês
        data_iteracao += relativedelta(months=1)

# --- Configurações de Download ---
ANO_INICIO_DOWNLOAD = 2010
MES_INICIO_DOWNLOAD = 1 # Janeiro

# O sufixo que identifica o tipo de dado na URL e no nome do arquivo.
# Pelo seu exemplo, "Bancos" é para Balancetes de Bancos.
# Se houver outros tipos de dados com padrões semelhantes,
# você pode chamar a função novamente com um tipo_dado diferente.
TIPO_DADO_PARA_BAIXAR = "Sociedades"

# Chama a função para iniciar o download
baixar_dados_bcb_mes_a_mes(
    ano_inicio=ANO_INICIO_DOWNLOAD,
    mes_inicio=MES_INICIO_DOWNLOAD,
    tipo_dado=TIPO_DADO_PARA_BAIXAR
)

print("\nProcesso de download concluído.")

Diretório 'dados_bcb_cosif_sociedades' já existe.
Iniciando download dos dados de 01/2010 até 07/2025...
Tentando baixar: 201001Sociedades.zip
  Arquivo '201001Sociedades.zip' já existe. Pulando.
Tentando baixar: 201002Sociedades.zip
  Arquivo '201002Sociedades.zip' já existe. Pulando.
Tentando baixar: 201003Sociedades.zip
  Arquivo '201003Sociedades.zip' já existe. Pulando.
Tentando baixar: 201004Sociedades.zip
  Arquivo '201004Sociedades.zip' já existe. Pulando.
Tentando baixar: 201005Sociedades.zip
  Arquivo '201005Sociedades.zip' já existe. Pulando.
Tentando baixar: 201006Sociedades.zip
  Arquivo '201006Sociedades.zip' já existe. Pulando.
Tentando baixar: 201007Sociedades.zip
  Arquivo '201007Sociedades.zip' já existe. Pulando.
Tentando baixar: 201008Sociedades.zip
  Arquivo '201008Sociedades.zip' já existe. Pulando.
Tentando baixar: 201009Sociedades.zip
  Arquivo '201009Sociedades.zip' já existe. Pulando.
Tentando baixar: 201010Sociedades.zip
  Arquivo '201010Sociedades.zip' já ex

In [ ]:
import zipfile
import os

diretorio_zips="dados_bcb_cosif_sociedades"
diretorio_destino_extraidos="dados_sociedades"

arquivos_no_diretorio = os.listdir("dados_bcb_cosif_sociedades")
zips_encontrados = [f for f in arquivos_no_diretorio if f.endswith('.zip')]

for i, nome_zip in enumerate(zips_encontrados):
    caminho_completo_zip = os.path.join("dados_bcb_cosif_sociedades", nome_zip)
    
    try:
        with zipfile.ZipFile(caminho_completo_zip, 'r') as zip_ref:
            # Extrai todo o conteúdo do zip para o diretório de destino
            zip_ref.extractall(diretorio_destino_extraidos)
            print(f"  Extraído '{nome_zip}' ({i+1}/{len(zips_encontrados)}) para '{diretorio_destino_extraidos}'.")
    except zipfile.BadZipFile:
        print(f"  Erro: '{nome_zip}' não é um arquivo zip válido ou está corrompido. Pulando.")
    except Exception as e:
        print(f"  Erro inesperado ao extrair '{nome_zip}': {e}. Pulando.")

DIRETORIO_DOS_ZIPS = "dados_bcb_cosif_sociedades"

DIRETORIO_DOS_EXTRAIDOS = "dados_sociedades"

  Extraído '201512Sociedades.zip' (1/182) para 'dados_sociedades'.
  Extraído '202203Sociedades.zip' (2/182) para 'dados_sociedades'.
  Extraído '201906Sociedades.zip' (3/182) para 'dados_sociedades'.
  Extraído '202003Sociedades.zip' (4/182) para 'dados_sociedades'.
  Extraído '201401Sociedades.zip' (5/182) para 'dados_sociedades'.
  Extraído '201111Sociedades.zip' (6/182) para 'dados_sociedades'.
  Extraído '201209Sociedades.zip' (7/182) para 'dados_sociedades'.
  Extraído '201212Sociedades.zip' (8/182) para 'dados_sociedades'.
  Extraído '202102Sociedades.zip' (9/182) para 'dados_sociedades'.
  Extraído '202103Sociedades.zip' (10/182) para 'dados_sociedades'.
  Extraído '201403Sociedades.zip' (11/182) para 'dados_sociedades'.
  Extraído '201005Sociedades.zip' (12/182) para 'dados_sociedades'.
  Extraído '201502Sociedades.zip' (13/182) para 'dados_sociedades'.
  Extraído '202112Sociedades.zip' (14/182) para 'dados_sociedades'.
  Extraído '201503Sociedades.zip' (15/182) para 'dados_so

In [ ]:
import pandas as pd
import os

def combinar_csvs(
    diretorio_csvs="dados_sociedades",
    nome_arquivo_saida="cosif_sociedades_financeiras.csv",
    diretorio_saida="dados_bcb_cosif_combinados"
):
    """
    Combina todos os arquivos CSV de um diretório em um único arquivo CSV.

    Args:
        diretorio_csvs (str): O caminho para o diretório onde os arquivos CSV estão.
        nome_arquivo_saida (str): O nome do arquivo CSV de saída combinado.
        diretorio_saida (str): O diretório onde o arquivo combinado será salvo.
    """
    print(f"Iniciando a combinação dos arquivos CSV de: {diretorio_csvs}")

    # Cria o diretório de saída para o arquivo combinado, se não existir
    if not os.path.exists(diretorio_saida):
        os.makedirs(diretorio_saida)
        print(f"Diretório '{diretorio_saida}' criado para o arquivo combinado.")
    else:
        print(f"Diretório '{diretorio_saida}' já existe.")

    # Lista todos os arquivos no diretório de CSVs
    arquivos_no_diretorio = os.listdir(diretorio_csvs)
    csv_encontrados = [f for f in arquivos_no_diretorio if f.endswith('.CSV')]

    if not csv_encontrados:
        print(f"Nenhum arquivo CSV encontrado no diretório '{diretorio_csvs}'.")
        return

    print(f"Encontrados {len(csv_encontrados)} arquivos CSV para combinar.")

    lista_dataframes = []

    for i, nome_csv in enumerate(csv_encontrados):
        caminho_completo_csv = os.path.join(diretorio_csvs, nome_csv)
        
        try:

            df = pd.read_csv(
                caminho_completo_csv,
                sep=';',              # Separador de colunas é ponto e vírgula
                encoding='latin1',    # Codificação de caracteres comum para o português
                skiprows=4,           # Pula as 4 primeiras linhas de metadados
                decimal=',',          # Usa vírgula como separador decimal para números
                on_bad_lines='warn'   # Avisa sobre linhas problemáticas, mas tenta continuar
            )

            lista_dataframes.append(df)
            print(f"  Lido '{nome_csv}' ({i+1}/{len(csv_encontrados)})")
        except FileNotFoundError:
            print(f"  Erro: Arquivo '{nome_csv}' não encontrado. Pulando.")
        except pd.errors.EmptyDataError:
            print(f"  Aviso: Arquivo '{nome_csv}' está vazio. Pulando.")
        except Exception as e:
            print(f"  Erro inesperado ao ler '{nome_csv}': {e}. Pulando.")

    if not lista_dataframes:
        print("Nenhum DataFrame válido para combinar.")
        return

    # Concatena todos os DataFrames na lista em um único DataFrame
    print("\nCombinando todos os DataFrames...")
    df_combinado = pd.concat(lista_dataframes, ignore_index=True)

    # Caminho completo para o arquivo de saída
    caminho_saida_completo = os.path.join(diretorio_saida, nome_arquivo_saida)

    # Salva o DataFrame combinado em um novo arquivo CSV
    # Você pode escolher o separador e o encoding para o arquivo de saída também.
    df_combinado.to_csv(caminho_saida_completo, sep=';', encoding='latin1', index=False)
    print(f"\nTodos os CSVs foram combinados em: '{caminho_saida_completo}'")

# --- Configurações de Combinação ---
# O diretório onde os arquivos CSV foram extraídos (o mesmo usado no script de extração)
DIRETORIO_DOS_CSVS_EXTRAIDOS = "dados_sociedades"

# O nome do arquivo CSV final combinado
NOME_ARQUIVO_CSV_FINAL = "cosif_sociedades_financeiras.csv"

# O diretório onde o arquivo combinado será salvo
DIRETORIO_ONDE_SALVAR_COMBINADO = "dados_bcb_cosif_combinados"

# Chama a função para iniciar a combinação
combinar_csvs(
    diretorio_csvs=DIRETORIO_DOS_CSVS_EXTRAIDOS,
    nome_arquivo_saida=NOME_ARQUIVO_CSV_FINAL,
    diretorio_saida=DIRETORIO_ONDE_SALVAR_COMBINADO
)


Iniciando a combinação dos arquivos CSV de: dados_sociedades
Diretório 'dados_bcb_cosif_combinados' já existe.
Encontrados 181 arquivos CSV para combinar.
  Lido '201305SOCIEDADES.CSV' (1/181)
  Lido '202207SOCIEDADES.CSV' (2/181)
  Lido '201801SOCIEDADES.CSV' (3/181)
  Lido '201212SOCIEDADES.CSV' (4/181)
  Lido '201311SOCIEDADES.CSV' (5/181)
  Lido '201104SOCIEDADES.CSV' (6/181)
  Lido '201702SOCIEDADES.CSV' (7/181)
  Lido '202307SOCIEDADES.CSV' (8/181)
  Lido '202206SOCIEDADES.CSV' (9/181)
  Lido '201902SOCIEDADES.CSV' (10/181)
  Lido '202407SOCIEDADES.CSV' (11/181)
  Lido '201510SOCIEDADES.CSV' (12/181)
  Lido '201101SOCIEDADES.CSV' (13/181)
  Lido '201608SOCIEDADES.CSV' (14/181)
  Lido '202010SOCIEDADES.CSV' (15/181)
  Lido '201606SOCIEDADES.CSV' (16/181)
  Lido '202201SOCIEDADES.CSV' (17/181)
  Lido '202101SOCIEDADES.CSV' (18/181)
  Lido '202412SOCIEDADES.CSV' (19/181)
  Lido '202103SOCIEDADES.CSV' (20/181)
  Lido '201602SOCIEDADES.CSV' (21/181)
  Lido '201603SOCIEDADES.CSV' (22/1